In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mybiotools as mbt
import pr_peaks
import os

# 2018-03-12 Preparing MB lab meeting
I want to get some data ready to be shown at MB's lab meeting.

In [ ]:
# general variables
pr_peaks_root_dir = '%s/work/CRG/projects/pr_peaks'%(os.getenv('HOME'))
figures_dir = '%s/figures'%(pr_peaks_root_dir)

In [ ]:
# general simulation parameters
nsteps = 100000
n = 12

# init site_taus
Hsites = [5,6]
Lsites = [i for i in xrange(n) if i not in Hsites]
site_taus = 2.0*np.ones(n)
site_taus[Hsites] = 5.0

# "null" contact model
Hnull = [[] for i in xrange(n)]

# the "contacts" model
H = [[] for i in xrange(n)]
H[Hsites[0]] = [Hsites[1]]
H[Hsites[1]] = [Hsites[0]]
boost = 2.0

# init the Jumping Models
uniform = pr_peaks.JumpingModel(Hnull,site_taus,None)
contact = pr_peaks.JumpingModel(H,site_taus,boost)

# values of the number of searchers
mus = np.arange(1,10,1)

# do the simulations
np.random.seed(85498)
samples = {}
for mu in mus :
    mbt.log_message('Uniform','mu = %d'%(mu))
    # init omega_t
    omega_t_initial = pr_peaks.init_omega_t(n,mu)
    uniform.run(nsteps,omega_t_initial)
    mbt.log_message('Uniform','mu = %d'%(mu))
    contact.run(nsteps,omega_t_initial)
    
# aftermath
pr_peaks.H_to_L(uniform,Hsites,Lsites)
pr_peaks.H_to_L(contact,Hsites,Lsites)

In [ ]:
xticklabels = ["L" for i in xrange(n)]
for Hsite in Hsites :
    xticklabels[Hsite] = "H"
fig,ax = plt.subplots(1,1,figsize=(10,4))
mbt.ax_only_y(ax,show_xaxis=True)
ax.bar(Lsites,site_taus[Lsites],color='b')
ax.bar(Hsites,site_taus[Hsites],color='r')
ax.set_ylabel("Site times",fontsize=32)
ax.set_xticks(range(n))
ax.set_xticklabels(xticklabels,)
plt.tick_params(axis='x',length=0)
fig.tight_layout()
fig.savefig('%s/LM_MB_site_affinities.pdf'%(figures_dir))
plt.show()

In [ ]:
xticklabels = ["L" for i in xrange(n)]
for Hsite in Hsites :
    xticklabels[Hsite] = "H"
    
for mu in [1,5,7] :
    fig,ax = plt.subplots(1,1,figsize=(10,4))
    mbt.ax_only_y(ax,show_xaxis=True)
    ax.bar(Lsites,uniform.theta[mu][Lsites],color='b')
    ax.bar(Hsites,uniform.theta[mu][Hsites],color='r')
    ax.set_ylabel("Occupancy",fontsize=32)
    ax.set_xticks(range(n))
    ax.set_xticklabels(xticklabels,)
    ax.text(0.9,0.8,"M = %d"%mu,transform=ax.transAxes,fontsize=24)
    plt.tick_params(axis='x',length=0)
    fig.tight_layout()
    fig.savefig('%s/LM_MB_occupancy_M-%d.pdf'%(figures_dir,mu))
    plt.show()

In [ ]:
fig = plt.figure(figsize=(6,4))
plt.plot(mus,uniform.H_to_L,'o--',color='b')
plt.xlabel(r'Number of searchers',fontsize=32)
plt.ylabel('H to L ratio',fontsize=32)
fig.tight_layout()
fig.savefig('%s/LM_MB_H_to_L_uniform.pdf'%(figures_dir))
plt.show()

In [ ]:
fig = plt.figure(figsize=(6,4))
plt.plot(mus,contact.H_to_L,'o--',color='b',label='HH model')
plt.plot(mus,uniform.H_to_L,'o--',color='r',label='Uniform model')
plt.xlabel(r'$\mu$')
plt.ylabel('H to L ratio')
plt.legend(loc='upper right')
plt.show()